## 2 - Cleaning the dataset 

In [1]:
## import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from IPython.display import display

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn import preprocessing

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [2]:
## loading data
df = pd.read_csv('data/train.csv')
df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [3]:
##save id to id datafram
df_id = df.loc[:,'ID']

## removing the 'Segmentation' column which is what we are trying to predict and ID
df = df.drop(['Segmentation'], axis= 'columns')

## convert all column names to lower case for ease of typing
df.columns = df.columns.str.lower()

## Cleaning the dataset

- 1) dealing with the missing values<br>
We will replace the missing values with the median as they are not affected by outliers for numerical variables, and the mode for categorical variables.

- 2) replace outliers<br>
We will cap the outliers at the 5% lower bound of 95% upper end of the dataset distribution. So any outliers beyond those quantiles will be replaced by the Q(5) and Q(95).

- 3) cleaning columns<br>
We will rename is to anon_cat (anonymised category) and keep the number of category: 1 instead of cat_1.\
We will also convert float family_size ot integer as a person is whole. Same for work experience as no decimal numbers.

- 4) encoding all categorical variables<br>


- 5) standarising the numerical variables<br>
Making sure the scales are consistent and more of a gaussian distribution since they are right-heavy

### 1) dealing with the missing values


In [4]:
## 1) dealing with the missing values

#assign median for numerical variables
df['work_experience'].fillna(int(df['work_experience'].median()), inplace=True)
df['family_size'].fillna(int(df['family_size'].median()), inplace=True)

#assign mode for categorical variables
df['ever_married'].fillna(df['ever_married'].mode()[0], inplace=True)
df['graduated'].fillna(df['graduated'].mode()[0], inplace=True)
df['profession'].fillna(df['profession'].mode()[0], inplace=True)
df['var_1'].fillna(df['var_1'].mode()[0], inplace=True)

In [5]:
## check all is filled
df.isna().sum()

id                 0
gender             0
ever_married       0
age                0
graduated          0
profession         0
work_experience    0
spending_score     0
family_size        0
var_1              0
dtype: int64

### 2) replace outliers


In [6]:
# below Q5 then above Q95

# age 
q1 = df['age'].quantile(0.05)
df['age'][df['age']<=q1] = q1


q4 = df['age'].quantile(0.95)
df['age'][df['age']>=q4] = q4

# work experience 
q1 = df['work_experience'].quantile(0.05)
df['work_experience'][df['work_experience']<=q1] = q1

q4 = df['work_experience'].quantile(0.95)
df['work_experience'][df['work_experience']>=q4] = q4

# family size
q1 = df['family_size'].quantile(0.05)
df['family_size'][df['family_size']<=q1] = q1

q4 = df['family_size'].quantile(0.95)
df['family_size'][df['family_size']>=q4] = q4

/var/folders/j8/vzdf09mx34vcvn_f8vhtx07w0000gn/T/ipykernel_69273/983151942.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['age'][df['age']<=q1] = q1
/var/folders/j8/vzdf09mx34vcvn_f8vhtx07w0000gn/T/ipykernel_69273/983151942.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['age'][df['age']>=q4] = q4
/var/folders/j8/vzdf09mx34vcvn_f8vhtx07w0000gn/T/ipykernel_69273/983151942.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [7]:
## checking outliers have been removed
df.describe()

#or with boxplots ( to uncomment)
#for column in num_col:
    #plt.figure(figsize=(10,2))
    #sns.boxplot(data=df, x=column, showfliers= True) # set showfliers to False to remove outliers

,id,age,work_experience,family_size
count,8068.000000,8068.000000,8068.000000,8068.000000
mean,463479.214551,43.254958,2.385473,2.765865
std,2595.381232,15.849885,3.032512,1.278922
min,458982.000000,21.000000,0.000000,1.000000
25%,461240.750000,30.000000,0.000000,2.000000
50%,463472.500000,40.000000,1.000000,3.000000
75%,465744.250000,53.000000,4.000000,4.000000
max,467974.000000,77.000000,9.000000,5.000000


### 3) cleaning columns

In [8]:
## cleaning of var_1
# renaming it to anon_cat for anonymised category
df.rename(columns={"var_1": "anon_cat"}, inplace=True)

# extracting the numbers from the string: 1 instead of cat_1
df['anon_cat'] = df['anon_cat'].str.extract('(\d+)')

# converting 'Family_Size' from float to int as the number of human is a whole number
df['family_size'] = df['family_size'].astype(int)


In [9]:
## checking that work_expeirence values are all whole numbers
we = df['work_experience']%1 == 0
we.value_counts() ## all non decimal numbers

## converting float to integer type
df['work_experience'] = df['work_experience'].astype(int)

### 4) encoding all categorical variables

In [10]:
## one hot encoding: columns that have binary values 

## gender 
gender_ohe = preprocessing.LabelEncoder()
df['gender'] = gender_ohe.fit_transform(df['gender'])


## ever_married
ever_married_ohe = preprocessing.LabelEncoder()
df['ever_married'] = ever_married_ohe.fit_transform(df['ever_married'])

## graduated
graduated_ohe = preprocessing.LabelEncoder()
df['graduated'] = graduated_ohe.fit_transform(df['graduated'])



In [11]:
## multi-categories encoding

## ordered category: spending_score
spending_score_oe = OrdinalEncoder()
df['spending_score'] = spending_score_oe.fit_transform(df['spending_score'].values.reshape(-1,1))

## unordered category: profession
## get dummy variables for 'profession' variable 
dummies = pd.get_dummies(df.profession, dtype=int)

## concatenate dummy variables to main df
df = pd.concat([df,dummies], axis='columns')

## drop the 'profession' and 1 dummy variable 'Artist' to avoid multicollinearity
df = df.drop(['profession','Artist'], axis = 'columns')



In [12]:
df.head()

,id,gender,ever_married,age,graduated,work_experience,spending_score,family_size,anon_cat,Doctor,Engineer,Entertainment,Executive,Healthcare,Homemaker,Lawyer,Marketing
0,462809,1,0,22,0,1,2.0,4,4,0,0,0,0,1,0,0,0
1,462643,0,1,38,1,1,0.0,3,4,0,1,0,0,0,0,0,0
2,466315,0,1,67,1,1,2.0,1,6,0,1,0,0,0,0,0,0
3,461735,1,1,67,1,0,1.0,2,6,0,0,0,0,0,0,1,0
4,462669,0,1,40,1,1,1.0,5,6,0,0,1,0,0,0,0,0


### 5) standardising the numerical categories



In [13]:
num_df = df[['age','family_size','anon_cat']]
cat_df = df.drop(['age','family_size','anon_cat'], axis='columns')

In [14]:
## create a scaler 
scaler = StandardScaler()

## standardise 
num_df = scaler.fit_transform(num_df.to_numpy())

## convert series to pandas data frame
num_df_scaled = pd.DataFrame(num_df, columns=['age','family_size','anon_cat'])

## concatenate scaled numerical variables and categorical variables
final_df = pd.concat([cat_df, num_df_scaled ], axis = 'columns')



In [15]:
## check final df
final_df

,id,gender,ever_married,graduated,work_experience,spending_score,Doctor,Engineer,Entertainment,Executive,Healthcare,Homemaker,Lawyer,Marketing,age,family_size,anon_cat
0,462809,1,0,0,1,2.0,0,0,0,0,1,0,0,0,-1.341100,0.965040,-0.808370
1,462643,0,1,1,1,0.0,0,1,0,0,0,0,0,0,-0.331566,0.183083,-0.808370
2,466315,0,1,1,1,2.0,0,1,0,0,0,0,0,0,1.498214,-1.380831,0.600711
3,461735,1,1,1,0,1.0,0,0,0,0,0,0,1,0,1.498214,-0.598874,0.600711
4,462669,0,1,1,1,1.0,0,0,1,0,0,0,0,0,-0.205374,1.746997,0.600711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8063,464018,1,0,0,0,2.0,0,0,0,0,0,0,0,0,-1.341100,1.746997,-2.921991
8064,464685,1,0,0,3,2.0,0,0,0,1,0,0,0,0,-0.520854,0.965040,-0.808370
8065,465406,0,0,1,1,2.0,0,0,0,0,1,0,0,0,-0.647045,-1.380831,0.600711
8066,467299,0,0,1,1,2.0,0,0,0,0,1,0,0,0,-1.025620,0.965040,0.600711


In [19]:
## saved final_df to csv 
# to be reused next in segmentation using Kmeans
final_df.to_csv('final_df.csv', index=False)


## Next: go to 3_kmeans.ipynb